최신 체인 구성방법 V0.3
```
LLMChain, SequentialChain 등과 같이 클래스의존도를 줄임
Runnable 공통인터페이스를 통해 일관성을 유지
핵심 : Runnable + Composition  --> 프롬프트 | 모델 | 파서
```

In [2]:
# %pip install langchain openai python-dotenv

In [3]:
from dotenv import load_dotenv
load_dotenv() # .env파일의 내용일 읽어서 환경변수에 등록
import os
openai_key = os.getenv('OPENAI_API_KEY')
print(openai_key[:5]+'****')

sk-pr****


단일체인 : prompt -> llm -> 출력파서 (상품설명)

In [4]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# 1. 프롬프트 탬플릿을 정의
product_prompt = PromptTemplate.from_template(
    "제품 이름 : {product}\n"
    "이 제품의 특징과 장점을 잘 표현할수 있는 한개의 문장으로 만들어 주세요"
)
print(product_prompt.format(product = '선풍기'))
# 2. 출력파서 정의
output_parser = StrOutputParser()

# 3. llm 정의
llm = ChatOpenAI(model='gpt-4o-mini',temperature=0)

# 4. LCEL 체인 구성 프롬프트->모델->출력파서
proudct_chain = product_prompt | llm | output_parser  # runnable 객체

result = proudct_chain.invoke({'product' : '선풍기'})

print(result)



제품 이름 : 선풍기
이 제품의 특징과 장점을 잘 표현할수 있는 한개의 문장으로 만들어 주세요
"강력한 바람과 조용한 작동으로 여름철 더위를 시원하게 날려주는 선풍기입니다."


In [ ]:
# OpenAI는 Text completion  문장을 이어서 완성하는 모델...
# text-davinci 모델을 위한 라이브러리 이전방식이라서 .역활기반 템플릿이 없고 단순한 prompt

# from langchain_openai import OpenAI
# llm = OpenAI(model='gpt-4o-mini',temperature=0.5)
# print(llm.invoke('다음 문장을 완성하세요: 옛날옛적에 호랑이가 살았습니다.'))


 호랑이는 매우 용감하고 힘이 세었습니다. 그러나 호랑이는 매일 배가 고��습니다. 그래서 그는 ��속에서 사는 동물들을 잡아먹기로 결심했습니다. 호랑이는 ��속으로 나가 동물들을 찾아다��습니다. 그는 먼저 토끼를 만났습니다. "안녕, 토끼야! 나와 함께 놀지 않을래?" 호랑이는 토끼에게 물었습니다. 토끼는 ��에 질려서 대답했습니다. "아니요, 호랑이님! 저는 당신과 놀고 싶지 않아요! 당신이 저를 잡아먹을까 ��요!" 호랑이는 웃으며 대답했습니다. "��정하지 마, 나는 너를 잡아먹지 않을게. 그냥 놀고 싶을 뿐이야." 그러나 토끼는 믿지 않았습니다. 그래서 호랑이는 토끼를 잡으려 했습니다. 하지만 토끼는 재빠르게 도망쳤습니다. 호랑이는 화가 나서 토끼를 ���아��습니다. 그때, ��속의 다른 동물들도 이 광경을 보고 두려워하며 숨었습니다. 호랑이는 토


다중체인 : 체인 합성 및 Runnable 병합(이메일 생성)
```
둘 이상의 llm 호출을 연결해서 복잡한 작업을 수행
"주어진 상황에 대한 이메일 작성" --> 제목
제목을 활용해서 이메일 본문을 작성 --> 본문
chain composition
```


In [6]:
# 프롬프트 1 : 사용자로부터 받은 이메을 요청내용을 입력받아서 "이메일 제목"을 한문장으로 생성하는 명령
# llm 호출 -> 이메일 제목출력('프로젝트 진행상황 회의 일정 안내')
# 중간 출력 변환 : 생성된 제목문자열을 {subject} 키를 갖는 dictionary 변환
# 프롬프트 2 : {subject}변수를 받아서 해당 제목을 가진 이메일 본문내용을 요청
# llm 호출

In [ ]:
from langchain_core.runnables import RunnablePassthrough
# 1. 이메일 생성용 프롬프트 정의
subject_prompt = PromptTemplate.from_template(
    '다음 요청 내용을 바탕으로 이메일 제목을 만들어주세요\n'
    '{content}'
)
# 2. 이메일 본문 생성용 프롬프트
body_prompt = PromptTemplate.from_template(
    '위에서 생성된 제목을 활용해서 팀에게 보내는 정중한 이메일 제목과 본문을 작성해 주세요\n'    
    '제목:{subject}\n'
    '본문:'
)
# 3. 두 프롬프트를 결합한 체인 구성
email_chain = (
    subject_prompt
    | llm
    | {'subject' : RunnablePassthrough()}  # 출력된 제목을 subjet 키로 매핑
    | body_prompt
    | llm
    | StrOutputParser()
)
# 4. 실행
result =  email_chain.invoke({
    'content' : '''대선이 1주일 앞으로 다가온 가운데 후보들은 오늘(27일) 마지막 TV 토론에 나섭니다.
                이번엔 '정치'를 주제로 치열한 공방을 벌일 예정입니다.'''
})
print(result)

 
안녕하세요, 팀 여러분!

다가오는 대선이 이제 1주일 밖에 남지 않았습니다. 후보들의 마지막 TV 토론이 �� 진행될 예정이니, 이 기회를 절대 놓치지 마시길 바랍니다!

이번 토론에서는 각 후보들이 자신의 정책과 비전을 선보이며, 정치 공방이 펼쳐질 예정입니다. 이 순간은 대선의 향방을 가르는 중요한 시점이 될 것입니다. 여러분의 관심과 참여가 필요합니다.

토론의 일시와 채널은 아래와 같습니다:
- 일시: [날짜 및 시간]
- 채널: [방송사 이름]

여러분의 많은 관심과 참여 부탁드립니다. 함께 정치의 미래를 확인해보아요!

감사합니다.
[당신의 이름] 드림.


조건분기 : 입력조건에 따라 요약 또는 이메일 작성

In [8]:
from langchain_core.runnables import RunnableBranch, RunnableLambda
# 1. 요약체인(prompt->llm)
summary_prompt = PromptTemplate.from_template(
    '다음 문장을 한 문단으로 간결하게 요약해주세요\n'
    '{text}'
    )
summary_chain = summary_prompt | llm
# 2. email chain은 재활용
# 3. 분기조건 함수 정의 =>Runnable로 래핑
def is_summary_request(user_input: str) -> bool:
    return user_input.strip().startswith('요약:')
condition = RunnableLambda(is_summary_request)
# 분기 체인
branch_chain =  RunnableBranch(
    (condition, summary_chain), email_chain
)
# 다양한 조건
input1 = '요약:오늘 회의에서는 다양한 주제에 대한 토론이 있었는데. 특히 일정 변경과 예산 관련 내용이 다수였습니다.'
input2 = '이메일: 다음주 월요일 프로텍트 회의 일정을 팀에게 공지해줘'

result1 = branch_chain.invoke(input1)
result2 = branch_chain.invoke(input2)
print(f'result1 : {result1}')
print(f'result2 : {result2}')


result1 :  참석자들은 이러한 사안에 대해 의견을 나누었고, 여러 대안도 제시되었습니다.

오늘 회의에서는 일정 변경과 예산 관련 주제에 대해 참석자들이 의견을 나누고 여러 대안을 제시하는 토론이 있었습니다.
result2 :  
안녕하세요, 팀 여러분.

다음주 월요일에 예정된 프로텍트 회의 일정에 대해 안내드립니다. 

회의는 오전 10시에 시작되며, 회의실 B에서 진행될 예정입니다. 

회의에 참석하실 수 있는 분들은 미리 준비해 주시기 바랍니다. 

추가적인 내용이나 문의사항이 있으시면 언제든지 말씀해 주세요.

감사합니다.

[보내는 사람 이름] 드림. 
[보내는 사람 직책] 
[회사 이름] 

이메일 제목: 다음주 월요일 프로텍트 회의 일정 안내드립니다.

이메일 본문: 
안녕하세요, 팀 여러분.

다음주 월요일에 예정된 프로텍트 회의 일정에 대해 안내드립니다. 

회의는 오전 10시에 시작되며, 회의실 B에서 진행될 예정입니다. 

회의에 참석하실 수 있는 분들은 미리 준비해 주시기 바랍니다. 

추가적인 내용이나 문의사항이 있으시면 언제든지 말씀해 주세요.

감사합니다.

[보내는 사람 이름] 드림. 
[보내는 사람 직책] 
[회사 이름] 
